<a href="https://colab.research.google.com/github/ibssasimon/LaughableDisney/blob/master/Copy_of_Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: August 28th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [3]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

     |████████████████████████████████| 645kB 7.4MB/s 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [4]:
!nvidia-smi

Sun Oct 20 16:15:20 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [5]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 179Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 88.0Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 224Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 184Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 195Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 116Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 153Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [6]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
import time

run_nam = "run" + str(time.time())

print(run_nam)

file_name = "billboard_lyrics.txt"

run1571566040.1607893


If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [7]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name=run_nam,
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:10<00:00, 10.60s/it]


dataset has 1791247 tokens
Training...
[10 | 52.67] loss=2.78 avg=2.78
[20 | 98.24] loss=2.60 avg=2.69
[30 | 143.88] loss=2.88 avg=2.75
[40 | 189.36] loss=3.08 avg=2.84
[50 | 234.88] loss=2.89 avg=2.85
[60 | 280.38] loss=2.88 avg=2.85
[70 | 325.93] loss=2.57 avg=2.81
[80 | 371.43] loss=2.54 avg=2.78
[90 | 416.98] loss=2.82 avg=2.78
[100 | 462.48] loss=2.67 avg=2.77
[110 | 507.97] loss=2.50 avg=2.75
[120 | 553.50] loss=3.02 avg=2.77
[130 | 598.98] loss=2.81 avg=2.77
[140 | 644.47] loss=2.68 avg=2.77
[150 | 690.01] loss=2.76 avg=2.76
[160 | 735.52] loss=2.68 avg=2.76
[170 | 781.00] loss=2.42 avg=2.74
[180 | 826.50] loss=2.80 avg=2.74
[190 | 872.03] loss=2.68 avg=2.74
[200 | 917.50] loss=2.83 avg=2.74
======== SAMPLE 1 ========
 it with the people in the streets aint nothing that youve done to get me crazy cause i wanna be your friend when i get your money baby if its fair im your friend i got to give what to you baby baby i say so baby when i get your money baby babyif you got me high en

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name=run_nam)

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

In [0]:
# nothing

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name="run1571566040.1607893")

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [8]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name="run1571566040.1607893")

Loading checkpoint checkpoint/run1571566040.1607893/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1571566040.1607893/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [9]:
gpt2.generate(sess, run_name="run1571566040.1607893")

id like it to be your night lady id like to be your night ladyid like it to be your night ladyid like it to be your night ladyid like it to be your night ladyid like it to be your night ladyid like it to be your night ladyid like it to be your night lady 
 ive been in love with you my whole life ive been on the road ive been in love with you all my life ive been on the road ive been in love with you my whole life ive been on the road ive been in love with you my whole life ive been on the road ive been in love with you my whole life ive been on the road ive been in love with you my whole life ive been on the road ive been in love with you my whole life ive been on the road ive been in love with you my whole life ive been on the road ive been in love with you my whole life ive been on the road ive been in love with you my whole life ive been on the road ive been in love with you my whole life ive been on the road ive been in love with you my whole life ive been on the road ive been in l

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [14]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="hey jude",
              nsamples=5,
              batch_size=5,
              run_name="run1571566040.1607893"
              )

hey jude jones j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus j in a chorus
 i wanna be your lover girl im gonna make you beg believe me i wanna be your lover girl i wanna be your lover girl the way you kiss me i want you to make me wanna be your lover girl oh oh oh oh oh oh oh yes you do i wanna be your lover girl i wanna be your lover girl youre the one for me i wanna be your lover girl i wanna be your lover girl youre the one for mei wanna be your lover girl im gonna make you beg believe me i wanna be your lover girl i wanna be your lover girl the way you kiss me i want you to make me wanna be your lover girl oh oh oh oh oh oh yes you do i wanna be your lover girl i wanna be your lover girl youre the one for me i wanna be your lover girl i wanna be your lover girl youre the one
hey

In [21]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="The",
              nsamples=5,
              batch_size=5,
              run_name="run1571566040.1607893"
              )

The girl that i love shes so sweet and kind and she loves me so sweet and sweet and she loves me so sweet to me she knows how to make me feel my love and she knows how to make me crazy in my sleep i know how to make you feel my love and i know how to make you feel my love and i know how to make you feel my love and i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love i know how to make you feel my love

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M "large" model.

In [0]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [0]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [0]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.